# Get confusion matrix for the trained model

First, import required modules, and set plotting parameters

In [ ]:
import caffe
import glob
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import sys

# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
sys.path.insert(0, caffe_root + 'python')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
caffe.set_mode_cpu()
net = caffe.Net(caffe_root + 'models/hw2/deploy.prototxt',
                caffe_root + 'models/hw2/hw2_LL_tune_iter_1000.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))

transformer.set_mean('data', np.load(caffe_root + 'data/hw2/mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

In [ ]:
# set net to batch size of 20
net.blobs['data'].reshape(20,3,227,227)

In [ ]:
im_names = {}
for i in range(25):
    path = '../data/hw2/test_images/' + str(i+1) + '/*'
    im_names[i] = glob.glob(path)

Feed in the image (with some preprocessing) and classify with a forward pass.

In [ ]:
pred = {}
for i in range(25):
    print 'Now testing class', i+1, 'out of ', 25
    imData = np.zeros((20, 3, 227, 227))
    for j, imagePath in enumerate(im_names[i]):
        imData[j,:,:,:] = transformer.preprocess('data', caffe.io.load_image(imagePath))       
    net.blobs['data'].data[...] = imData
    out = net.forward()
    pred[i] = out['prob'][:20].argmax(axis = 1)

In [ ]:
def get_conf_mat(pred):
    conf_mat = np.zeros((25, 25))
    for i in pred:
        for j in range(pred[i].shape[0]):
            conf_mat[i, pred[i][j]] += 1
    conf_mat = conf_mat / np.sum(conf_mat, axis = 1)
    return conf_mat

In [ ]:
conf_mat = get_conf_mat(pred)
tick_marks = np.arange(25)
class_names = ['sax',
               'limousine',
               'gong',
               'shovel',
               'fiddler crab',
               'bearskin',
               'sea lion',
               'baseball',
               'meerkat',
               'puffer',
               'dalmatian',
               'Saluki',
               'barber chair',
               'church',
               'yurt',
               'obelisk',
               'bluetick',
               'lakeside',
               'mushroom',
               'measuring cup',
               'projectile',
               'Brittany spaniel',
               'ant',
               'snow leopard',
               'sandbar']

plt.xticks(tick_marks, class_names, rotation=90)
plt.yticks(tick_marks, class_names)    
plt.imshow(conf_mat , cmap=plt.cm.Blues)

In [ ]:
accs = [conf_mat[i, i] for i in range(25)]
acc = np.mean(accs)
print acc

In [ ]:
for i in range(25):
    print i+1, class_names[i], accs[i]